Code for implementing the distributed Source Localization experiments.

Parts of this code are taken from Fernando Gama, available at: https://github.com/alelab-upenn/graph-neural-networks.

## Importing 

In [ ]:
#Standard libraries:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle
import datetime
from copy import deepcopy

import torch; torch.set_default_dtype(torch.float64)
import torch.nn as nn
import torch.optim as optim

#GNNs libraries
import Utils.graphTools as graphTools
import Utils.dataTools
import Utils.graphML as gml
import Utils.graphAdaptiveActivations as gaActivations
import Modules.architectures as archit
import Modules.model as model
import Modules.train as train
import Modules.loss as loss

#Miscellaneous libraries
from Utils.miscTools import writeVarValues
from Utils.miscTools import saveSeed

## Setting Parameters

Let's set up the directors for writing the logs and the results from each run.

In [ ]:
graphType = 'SBM'
thisFilename = 'SourceLocalization' #This is the general name of all related files
saveDirRoot = 'experiments_sourceLocalization'
saveDir = os.path.join(saveDirRoot, thisFilename) # Dir where to save all the results from each run

We generate next a `.txt` file for saving the values used for the setting parameters for an easier reference. We append date and time as well for each of the runs, to avoid overwriting when multiple experiments are run. 

In [ ]:
today = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
saveDir = saveDir + '-' + graphType + '-' + today

# If the directory does not exist, create it
if not os.path.exists(saveDir):
    os.makedirs(saveDir)
    
# Create the file where the setting parameters will be saved.
varsFile = os.path.join(saveDir,'hyperparameters.txt')

with open(varsFile, 'w+') as file:
    file.write('%s\n\n' % datetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S"))

Save seeds for further reproducibility.

In [ ]:
#   PyTorch seeds
torchState = torch.get_rng_state()
torchSeed = torch.initial_seed()
#   Numpy seeds
numpyState = np.random.RandomState().get_state()
#   Collect all random states
randomStates = []
randomStates.append({})
randomStates[0]['module'] = 'numpy'
randomStates[0]['state'] = numpyState
randomStates.append({})
randomStates[1]['module'] = 'torch'
randomStates[1]['state'] = torchState
randomStates[1]['seed'] = torchSeed

saveSeed(randomStates, saveDir)


If GPU is available, we use it.

In [ ]:
useGPU = True 

## Data Parameters

We set the number of nodes in the SBM graph, as well as the number of classes (i.e., number of communities).

In [ ]:
nNodes = 40

In [ ]:
nClasses = 4

## Training Parameters

We set next the training options, such as optimizer, loss function, number of epochs, etc.

In [ ]:
trainer = 'ADAM' # Options: 'SGD', 'ADAM', 'RMSprop'
learningRate = 0.001 # In all options
beta1 = 0.9 # beta1 if 'ADAM', alpha if 'RMSprop'
beta2 = 0.999 # ADAM option only

In [ ]:
lossFunction = torch.nn.CrossEntropyLoss

In [ ]:
nEpochs = 800 # Number of epochs
batchSize = 100 # Batch size
doLearningRateDecay = False # Learning rate decay
learningRateDecayRate = 0.9 # Rate
learningRateDecayPeriod = 1 # How many epochs after which update the learning rate
validationInterval = 20 # How many training steps to do the validation

We save next the values we set.

In [ ]:
writeVarValues(varsFile,
               {'trainer': trainer,
                'learningRate': learningRate,
                'beta1': beta1,
                'beta2': beta2,
                'lossFunction': lossFunction,
                'nEpochs': nEpochs,
                'batchSize': batchSize,
                'doLearningRateDecay': doLearningRateDecay,
                'learningRateDecayRate': learningRateDecayRate,
                'learningRateDecayPeriod': learningRateDecayPeriod,
                'validationInterval': validationInterval})

## Architecture

We set the architecture we will use. Multiple architecture can be defined and included in the model list.

In [ ]:
modelList = []

We will employ the Local GNN defined by Fernando Gama (available at https://github.com/alelab-upenn/graph-neural-networks), together with our proposed max graph-adaptive activation function. We perform no pooling. The model has two layers.

In the end, we save the values.

In [ ]:
hParamsLclGNN = {} # Hyperparameters (hParams) for the Local GNN (LclGNN)

# Chosen architecture is the Local GNN
hParamsLclGNN['name'] = 'LclGNN'
hParamsLclGNN['archit'] = archit.LocalGNN
    
# Graph convolutional parameters
hParamsLclGNN['dimNodeSignals'] = [1, 5, 5] # Features per layer
hParamsLclGNN['nFilterTaps'] = [2, 2] # Number of filter taps per layer
hParamsLclGNN['bias'] = True # Decide whether to include a bias term

# Nonlinearity
hParamsLclGNN['nonlinearity'] = gaActivations.MaxGAActivation

# Pooling
hParamsLclGNN['poolingFunction'] = gml.NoPool # Summarizing function
hParamsLclGNN['nSelectedNodes'] = None # To be determined later on
hParamsLclGNN['poolingSize'] = [1, 1] # poolingSize-hop neighborhood that is affected by the summary
    
# Readout layer: local linear combination of features
hParamsLclGNN['dimReadout'] = [nClasses] # Dimension of the fully connected layers
        # after the GCN layers (map); this fully connected layer is applied only
        # at each node, without any further exchanges nor considering all nodes
        # at once, making the architecture entirely local.
        
# Graph structure
hParamsLclGNN['GSO'] = None # To be determined later on
hParamsLclGNN['order'] = None # Not used because there is no pooling

hParamsLclGNN2Ly = deepcopy(hParamsLclGNN)

hParamsLclGNN2Ly['name'] += '2Ly' # Name of the architecture

#Save Values:
writeVarValues(varsFile, hParamsLclGNN2Ly)
modelList += [hParamsLclGNN2Ly['name']]

## Logging

Decide on the logging variable, whether we want to print statemets during running and save variables.

In [ ]:
doPrint = True # Decide whether to print stuff while running
doLogging = True # Log into tensorboard
doSaveVars = True # Save (pickle) useful variables
doFigs = True # Plot some figures (this only works if doSaveVars is True)


#\\\ Save values:
writeVarValues(varsFile,
               {'doPrint': doPrint,
                'doSaveVars': doSaveVars})

## Setup

Determine the processing unit.

In [ ]:
if useGPU and torch.cuda.is_available():
    device = 'cuda:0'
    torch.cuda.empty_cache()
else:
    device = 'cpu'
    
# Notify:
if doPrint:
    print("Device selected: %s" % device)

## Training Options

We create a dictionary to pass to the train function with all the options set before.

In [ ]:
trainingOptions = {}

if doSaveVars:
    trainingOptions['saveDir'] = saveDir
if doPrint:
    trainingOptions['printInterval'] = printInterval
if doLearningRateDecay:
    trainingOptions['learningRateDecayRate'] = learningRateDecayRate
    trainingOptions['learningRateDecayPeriod'] = learningRateDecayPeriod
trainingOptions['validationInterval'] = validationInterval

## Data

We define a function for generating the data for the Source Localization problem on the SBM graph.

In [ ]:
def data_generation_SBM(G, sourceNodes, nodesPerCommunity, numberOfCommunities, tMax = None):
    # If no tMax is specified, set it the maximum possible.
    if tMax == None:
        tMax = G.N
    
    # Get the largest eigenvalue of the weighted adjacency matrix
    EW, VW = graphTools.computeGFT(G.W, order = 'totalVariation')
    eMax = np.max(EW)
    # Normalize the matrix so that it doesn't explode
    Wnorm = G.W / eMax
    
    
    # Since the signals are generated as W^t * delta, this reduces to the
    # selection of a column of W^t (the column corresponding to the source
    # node). Therefore, we generate an array of size tMax x N x N with all
    # the powers of the matrix, and then we just simply select the
    # corresponding column for the corresponding time
    lastWt = np.eye(G.N, G.N)
    Wt = lastWt.reshape([1, G.N, G.N])
    for t in range(1,tMax):
        lastWt = lastWt @ Wnorm
        Wt = np.concatenate((Wt, lastWt.reshape([1, G.N, G.N])), axis = 0)
    
    #We next define the signals and the labels (communities that generated the signal)
    signals = []
    labels = []
    
    for source in sourceNodes:
        
        source_label = 0
        
        for c in range(0, numberOfCommunities):
            if source in nodesPerCommunity[c]:
                source_label = c
                break
        
        for diffusionTime in range(0, tMax):
            signals.append(Wt[diffusionTime, :, source])
            labels.append(source_label)
    
    #Check shape
    print(Wt.shape)
    
    return signals, labels 

Let's define a function that splits the data in train, validation and test sets based on the indices we provide.

In [ ]:
def getData(signals, labels, train_indexes, validation_indexes, test_indexes, labelID):
    train_data = []
    train_labels = []
    train_labelIDs = []
    for tr_index in train_indexes:
        train_data.append(signals[tr_index])
        train_labels.append(labels[tr_index])
        train_labelIDs.append(labelID)
        
        
    validation_data = []
    validation_labels = []
    validation_labelIDs = []
    for val_index in validation_indexes:
        validation_data.append(signals[val_index])
        validation_labels.append(labels[val_index])
        validation_labelIDs.append(labelID)
        
    test_data = []
    test_labels = []
    test_labelIDs = []
    for tst_index in test_indexes:
        test_data.append(signals[tst_index])
        test_labels.append(labels[tst_index])
        test_labelIDs.append(labelID)
    
    #Check the split
    print(f"Train {len(train_data)}")
    print(f"Test {len(test_data)}")
    print(f"Validation {len(validation_data)}")
        
    
    #Generate the source localization data, given the split
    data = Utils.dataTools.SourceLocalization(np.asarray(train_data), np.asarray(train_labels), np.asarray(train_labelIDs),
                                          np.asarray(validation_data), np.asarray(validation_labels), np.asarray(validation_labelIDs),
                                          np.asarray(test_data), np.asarray(test_labels), np.asarray(test_labelIDs))

    data.astype(torch.float64)
    data.to(device)
    
    return data
    

## Graph

Read and check the shape of the graphs, perviously generated.

In [ ]:
with open ('graphs', 'rb') as fp:
    allGraphs = pickle.load(fp)
    
print(np.shape(allGraphs))

## Model Initialization And Training

We initialize the model previously defined.

In [ ]:
def initialize_model(thisModel, nNodes, S, order):
    
    # Get the corresponding parameter dictionary
    hParamsDict = deepcopy(eval('hParams' + thisModel))

    #Remove the 'name' and 'archit' fields from the dictionary,
    #as these do not need to be passed to the architecture.
    thisName = hParamsDict.pop('name')
    callArchit = hParamsDict.pop('archit')


    #Optimizer options
    thisTrainer = trainer
    thisLearningRate = learningRate
    thisBeta1 = beta1
    thisBeta2 = beta2

   
    # Graph Shift Operaror
    hParamsDict['GSO'] = S
    
    # Add the number of nodes for the no-pooling part
    if '1Ly' in thisName:
        hParamsDict['nSelectedNodes'] = [nNodes]
    elif '2Ly' in thisName:
        hParamsDict['nSelectedNodes'] = [nNodes, nNodes]

    #Architecture
    thisArchit = callArchit(**hParamsDict)
    thisArchit.to(device)

    #Optimizer
    if thisTrainer == 'ADAM':
        thisOptim = optim.Adam(thisArchit.parameters(),
                                   lr = learningRate,
                                   betas = (beta1, beta2)) #, weight_decay=1e-5)
    elif thisTrainer == 'SGD':
        thisOptim = optim.SGD(thisArchit.parameters(),
                                  lr = learningRate)
    elif thisTrainer == 'RMSprop':
        thisOptim = optim.RMSprop(thisArchit.parameters(),
                                      lr = learningRate, alpha = beta1)

    #Loss
    thisLossFunction = loss.adaptExtraDimensionLoss(lossFunction)
    #thisLossFunction = lossFunction

    #Model
    modelCreated = model.Model(thisArchit,
                                   thisLossFunction,
                                   thisOptim,
                                   thisName, saveDir, order)
    

    writeVarValues(varsFile,
                       {'name': thisName,
                        'thisTrainer': thisTrainer,
                        'thisLearningRate': thisLearningRate,
                        'thisBeta1': thisBeta1,
                        'thisBeta2': thisBeta2})

        
    return modelCreated



Now that we initialized the model, we can perform the trianing.

In [ ]:
all_graph_accuracies = []

#Maximum time of diffusion
tMax = 30

cnt = 0
for G in allGraphs:
    print(f"GRAPH {cnt}")
    pred_node_accuracies = []
        
    #We retrieve the source nodes, the nodes per community and the nodes for which we want to perform the prediction
    sourceNodes, nodesPerCommunity, prediction_nodes = graphTools.computeSourceNodes(G.W, nClasses)

    #Given these nodes, we generate the signals and labels
    signals, labels = data_generation_SBM(G, sourceNodes, nodesPerCommunity, nClasses, tMax)
    
    #Generate random permutation and retrieve the train, validation and test indexes
    indexes = np.arange(0, len(signals))
    perm_indexes = np.random.permutation(indexes)
    train_indexes, validation_indexes, test_indexes = np.split(perm_indexes, [int(.8 * len(perm_indexes)), int(.9 * len(perm_indexes))])

    #Used for file name
    cnt_pred = 0
    
    #We perform the training and evaluation for each of the nodes chosen for prediction
    for labelID in prediction_nodes:  
        
        #Generate the files for saving
        thisFilename = 'finiteTimeConsensus graph ' + str(cnt) + ' prediction node ' + str(cnt_pred) 
        saveDir2 = os.path.join(saveDir, thisFilename) # Director where to save all the results from each run
        #Create the director
        if not os.path.exists(saveDir2):
            os.makedirs(saveDir2)
        trainingOptions['saveDir'] = saveDir2
        
        #Get the data
        data = getData(signals, labels, train_indexes, validation_indexes, test_indexes, labelID)
        
        
        #Initialize the models dictionary
        modelsGNN = {}
        thisName = modelList[0]
        
       
        #Ordering
        G.computeGFT()
        S, order = graphTools.permIdentity(G.S/np.max(np.diag(G.E)))
        
        #Initialize the local GNN model        
        LocalGNN = initialize_model(modelList[0], nNodes, S, order)
        
        #Add it to the dictionary
        modelsGNN[thisName] = LocalGNN
        
        #Train the model
        train.MultipleModels(modelsGNN, data,
                     nEpochs = nEpochs, batchSize = batchSize,
                     **trainingOptions)
        
        #EVALUATION
        
        #Get the test samples
        xTest, yTest, idsTest = data.getSamples('test')
        xTest = xTest.unsqueeze(1)
        xTest = xTest.to(device)
        yTest = yTest.to(device)
    
    
    
        for key in modelsGNN.keys():
            
            with torch.no_grad():
                # Process the samples
                yHatTest = modelsGNN[key].archit.singleNodeForward(xTest, idsTest)
                # We compute the accuracy
                accuracy = data.evaluate(yHatTest, yTest)

        #Save the performance
        all_graph_accuracies.append(accuracy)
        
        #makePlots(saveDir, modelList)
        cnt_pred += 1
        
        
    cnt += 1
  


In [ ]:
#Print results
print(all_graph_accuracies)
print(f"mean {torch.mean(torch.tensor(all_graph_accuracies))}")
print(f"std {torch.std(torch.tensor(all_graph_accuracies))}")